In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.parser import parse


In [2]:
#Data directories
popdir = 'C:\\Users\\mlamherr\\Data'
savedir = 'C:\\Users\\mlamherr\\Data'

In [3]:
#Bus areas 1-16 in numerical order as case10K file
WA = ['AVA', 'BPAT','CHPD','DOPD','GCPD','PSEI','SCL','TPWR','AVA', 'BPAT']
OR = ['BPAT','GRID','IPCO','PACW','PGE']
CAnorth = ['BANC','CISO','PACW','BPAT']
CABayArea = ['CISO']
CAcentral = ['CISO', 'TIDC']
CAsw = ['CISO', 'LDWP']
CAse = ['CISO', 'IID']
NV = ['NEVP']
AZ = ['AZPS','DEAA','GRIF','GRMA','HGMA','PNM','SRP','TEPC','WALC']
UT = ['PACE']
NM = ['EPE','PNM']
CO = ['PSCO','WACM']
WY = ['PACE','WACM']
ID = ['AVA','BPAT','IPCO','PACE']
MT = ['BPAT','GWA','NWMT','WAUW','WWA'] 
ElPaso = ['EPE']

BA_all = sorted(list(set().union(WA,OR,CAnorth,CABayArea,CAcentral,CAsw,CAse,NV,AZ,UT,NM,CO,WY,ID,MT,ElPaso)))

#### CA regions (county groups)

* Northern California: Butte, Colusa, Del Norte, El Dorado, Glenn, Humboldt, Lake, Lassen, Mendocino, Modoc, Nevada, Placer, Plumas, Sacramento, Shasta, Sierra, Siskiyou, Sutter, Tehama, Trinity, Yolo, Yuba
* Bay Area: Alameda, Contra Costa, Marin, Napa, San Francisco, San Mateo, Santa Clara, Santa Cruz, Solano, Sonoma
* Central California: Alpine, Amador, Calaveras, Fresno, Inyo, Kings, Madera, Mariposa, Merced, Mono, Monterey, San Benito, San Joaquin, Stanislaus, Tulare, Tuolumne
* Southeast California: Imperial, Orange, Riverside, San Bernardino, San Diego
* Southwest California: Kern, Los Angeles, San Luis Obispo, Santa Barbara, Ventura

In [4]:
Northern_California = ['Butte','Colusa','Del Norte','El Dorado','Glenn','Humboldt',
                       'Lake','Lassen','Mendocino','Modoc','Nevada','Placer','Plumas',
                       'Sacramento','Shasta','Sierra','Siskiyou','Sutter','Tehama',
                       'Trinity','Yolo','Yuba']
Bay_Area = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo',
            'Santa Clara','Santa Cruz','Solano','Sonoma']
Central_California = ['Alpine','Amador','Calaveras','Fresno','Inyo','Kings','Madera',
                      'Mariposa','Merced','Mono','Monterey','San Benito','San Joaquin',
                      'Stanislaus','Tulare','Tuolumne']
Southeast_California = ['Imperial','Orange','Riverside','San Bernardino','San Diego']
Southwest_California = ['Kern','Los Angeles','San Luis Obispo','Santa Barbara','Ventura']

CA = [Northern_California, Bay_Area, Central_California, Southeast_California, Southwest_California]
CAnames = ['Northern_California', 'Bay_Area', 'Central_California', 'Southeast_California', 'Southwest_California']
CA_all = Northern_California + Bay_Area + Central_California + Southeast_California +Southwest_California
CA_union = list(set().union(Northern_California, Bay_Area, Central_California, Southeast_California, Southwest_California))
shortCAnames = ['CAnorth','CABayArea','CAcentral','CAsw','CAse']

numCounties = 0
for x in CA:
    print(len(x))
    numCounties = numCounties + len(x)
print(numCounties)
print(len(CA_all))
print(len(CA_union))

22
10
16
5
5
58
58
58


In [5]:
# CA regions: get populations of demand regions

CA_population_file = 'Web_ACS2016_Pop-Race.xlsx'
df = pd.read_excel(io = popdir + "/" + CA_population_file, sheet_name='Total Pop & Median Age', skiprows=[0,1,2,3,4])


In [6]:
df.head()

,Geography,Estimate,Margin of Error,Estimate.1,Margin of Error.1,Unnamed: 5,Summary Level,County,Place
0,California,39250017,*****,36.4,0.2,NaN,40,NaN,NaN
1,Alameda County,1647704,*****,37.4,0.2,NaN,50,1.0,NaN
2,Alameda city,78907,29,42.2,2.1,NaN,160,1.0,562.0
3,Berkeley city,121241,34,32.1,1.6,NaN,160,1.0,6000.0
4,Castro Valley CDP,65153,4863,43.1,2.9,NaN,160,1.0,11964.0


In [7]:
df.columns

Index(['Geography', 'Estimate', 'Margin of Error', 'Estimate.1',
       'Margin of Error.1', 'Unnamed: 5', 'Summary Level', 'County', 'Place'],
      dtype='object')

In [8]:
df.drop(columns =['Margin of Error', 'Estimate.1',
       'Margin of Error.1', 'Unnamed: 5', 'Summary Level', 'County', 'Place'], inplace=True)

In [9]:
df.head()

,Geography,Estimate
0,California,39250017
1,Alameda County,1647704
2,Alameda city,78907
3,Berkeley city,121241
4,Castro Valley CDP,65153


In [10]:
counties = df.loc[df['Geography'].str.contains('County')][['Geography','Estimate']]
counties = counties.reset_index(drop=True)

In [11]:
counties['Geography'] = counties['Geography'].str.replace(' County','')
counties.head()

,Geography,Estimate
0,Alameda,1647704
1,Butte,226864
2,Contra Costa,1135127
3,El Dorado,185625
4,Fresno,979915


Some counties do not have population data (from this source) because they have less than 65000 residents. There are total of 58 counties but only 40 have >65000 in 2016.

In [12]:
intersection = set(counties['Geography']) & set(pd.Series(CA_union))
union = set(counties['Geography']) | set(pd.Series(CA_union))
print(set(union - intersection))  ## counties with < 65000 in 2016

{'Lassen', 'Trinity', 'Sierra', 'Colusa', 'Mono', 'Modoc', 'San Benito', 'Tuolumne', 'Calaveras', 'Mariposa', 'Amador', 'Plumas', 'Glenn', 'Alpine', 'Tehama', 'Inyo', 'Del Norte', 'Siskiyou'}


In [13]:
popcount = {}
for i in zip(shortCAnames, CA):
    region_name = i[0]
    region_county_list = i[1]

    popcount[region_name] = counties[counties['Geography'].isin(region_county_list)]['Estimate'].sum()

    

In [14]:
popcount

{'CAnorth': 3262336,
 'CABayArea': 7958684,
 'CAcentral': 3724007,
 'CAsw': 11199001,
 'CAse': 12601498}

In [15]:
#Add other population data cut and paste from other sources
#To do: The US Census data is automatable; not about El Paso yet
popcount['WA']=7280934
popcount['OR']=4085989
popcount['NV']=2939254
popcount['AZ']=6908642
popcount['UT']=3044321
popcount['NM']=2085432
popcount['CO']=5530105
popcount['WY']=584910
popcount['ID']=1680026
popcount['MT']=1038656
popcount['ElPaso']=678058

In [16]:
popcount

{'CAnorth': 3262336,
 'CABayArea': 7958684,
 'CAcentral': 3724007,
 'CAsw': 11199001,
 'CAse': 12601498,
 'WA': 7280934,
 'OR': 4085989,
 'NV': 2939254,
 'AZ': 6908642,
 'UT': 3044321,
 'NM': 2085432,
 'CO': 5530105,
 'WY': 584910,
 'ID': 1680026,
 'MT': 1038656,
 'ElPaso': 678058}

In [17]:
#ba2016 = pd.read_csv('')